# Use Case: ZAF043 Crowd Anomaly Detection

## 1 - Load the data

In [5]:
import os
import cv2
import numpy as np

def load_training_data(directory):
    # Initialize lists for frames and labels
    frames = []
    labels = []

    # Iterate over the files in the directory
    for filename in os.listdir(directory):
        # Get the full file path
        file_path = os.path.join(directory, filename)

        # Load the frame using OpenCV
        frame = cv2.imread(file_path)

        # Preprocess the frame if needed

        # Append the frame to the frames list
        frames.append(frame)

        # Extract the label from the filename (assuming the filename contains the label information)
        label = filename.split("_")[0]  # Extract the label from the filename based on the naming convention
        labels.append(label)

    # Convert frames and labels to numpy arrays
    frames = np.array(frames)
    labels = np.array(labels)

    return frames, labels

# Directory path containing the training data
training_data_directory = "D:\\Data Set\\Crowd Anomaly Detection\\UCSD_Anomaly_Dataset"

# Load training data
training_frames, training_labels = load_training_data(training_data_directory)

# Print the shape of the training frames and labels
print("Training Frames Shape:", training_frames.shape)
print("Training Labels Shape:", training_labels.shape)

Training Frames Shape: (6,)
Training Labels Shape: (6,)


## 2 - Data Preprocessing, Exploratory Data Analysis

In [6]:
def extract_features(frames):
    features = []
    
    # Convert frames to grayscale
    gray_frames = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in frames]
    
    # Calculate optical flow for consecutive frames
    for i in range(len(frames) - 1):
        prev_frame = gray_frames[i]
        next_frame = gray_frames[i + 1]
        
        # Compute optical flow using Lucas-Kanade method
        flow = cv2.calcOpticalFlowFarneback(prev_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        # Extract relevant features from the optical flow
        # Example: Compute the mean and standard deviation of the flow vectors
        mean_flow = np.mean(flow)
        std_flow = np.std(flow)
        
        # Append the extracted features to the list
        features.append([mean_flow, std_flow])
    
    return np.array(features)

## 3 - Models
In this code, the train_mdt_model() function takes features (the extracted features) and num_components (the number of components in the MDT model) as inputs. It initializes a Gaussian Mixture Model (GMM) with n_components=num_components and fits it to the features using the fit() method. The trained model is then returned.
The scikit-learn library installed (pip install scikit-learn) before running this code.

In [7]:
from sklearn.mixture import GaussianMixture

def train_mdt_model(features, num_components):
    # Train a Gaussian Mixture Model (GMM) on the extracted features
    gmm = GaussianMixture(n_components=num_components)
    gmm.fit(features)
    return gmm

In [12]:
def compute_anomaly_score(observation, model):
    # Compute the anomaly score by comparing the observation with the learned model
    score = -model.score_samples(observation.reshape(1, -1))
    return score

In [34]:
import os
import tifffile

# Specify the directory path containing the frames
directories =[
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train001",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train002",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train003",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train004",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train005",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train006",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train007",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train009",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train010",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train011",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train012",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train013",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train014",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train015",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train016",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train017",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train018",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train019",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train020",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train020",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train021",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train022",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train023",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train024",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train025",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train026",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train027",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train028",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train029",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train031",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train032",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train033",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train034"
    
]
# Initialize an empty list to store the frames
frames = []

# Iterate over the directories
for directory_path in directories:
    # Iterate over the files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Check if the file is a .tif image
        if filename.endswith(".tif"):
            # Read the frame using tifffile
            frame = tifffile.imread(file_path)

            # Append the frame to the list
            frames.append(frame)

# Print the number of frames loaded
print("Number of frames:", len(frames))

Number of frames: 7000


In [36]:
# Initialize variables
num_components = 5  # Number of components in the MDT model
features = []

In [38]:
def extract_features(frames):
    features = []

    # Calculate optical flow for consecutive frames
    for i in range(len(frames) - 1):
        frame1 = frames[i]
        frame2 = frames[i + 1]

        # Calculate optical flow
        flow = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Flatten the flow matrix and append it to the features
        flow_flat = flow.reshape(-1)
        features.append(flow_flat)

    return features

In [ ]:
# Create an empty list to store the features
features = []

# Iterate over the frames and extract features
for frame in frames:
    # Preprocess frame if needed
    # Extract features from the frame
    frame_features = extract_features(frame)
    features.append(frame_features)

In [39]:
# Convert features to numpy array
features = np.array(features)

In [42]:
# Train MDT model
mdt_model = train_mdt_model(features, num_components)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

### *Conclusion:* The model which can be recommended to the client is ... (Put here also essential recommendations to the client how to use your model).

## 4 - Saving the model
Save the DS best model in the Jupyter notebook `model.ipynb` in one of the following formats:

- `network.save('model.h5')` #keras
- `joblib.dump(model, "model.pkl")` # optional
- `torch.save(model.state_dict(), './model.pt')` #pytorch
- `model.save('path/to/model')`

End of document.